### Hugo Englund | 2020-12-19

# Laboration 6: Part B
### Outline
In this part of laboration we will implement a suitable performance measure for Q-learning, and then compare the best model from Part A with Gérons various Deep Q-learning neural networks (DQN):
1. DQN 
2. Double DQN
3. Dueling Double DQN

## Set up Colab environment

In [3]:
# Import needed libraries
import tensorflow as tf
print('TensorFlow version:', tf.__version__)

import tensorflow.keras
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.layers import Flatten
from tensorflow.keras.utils import to_categorical
print('Keras version:',tensorflow.keras.__version__)

# Helper libraries
import os
import numpy as np
import pandas as pd
import cv2
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from numpy import argmax

# Matlab plotting
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
# To plot pretty figures
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# To get smooth animations
import matplotlib.animation as animation
mpl.rc('animation', html='jshtml')
# For this Lab
import random
import gym

TensorFlow version: 2.4.0
Keras version: 2.4.0


In [4]:
# Test for GPU and determine what GPU we have
# Could also use https://github.com/anderskm/gputil
import sys
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    IN_COLAB = 'google.colab' in sys.modules
    if IN_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
else:
    !nvidia-smi -L

No GPU was detected. CNNs can be very slow without a GPU.
Go to Runtime > Change runtime and select a GPU hardware accelerator.


## Create environment

In [5]:
# Define some useful functions
def running_mean(x, N=10):
    """ Return the running mean of N element in a list
    """
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

In [6]:
# Create the Gym environment (CartPole)
env = gym.make('CartPole-v1')

print('Action space is:', env.action_space)
print('Observation space is:', env.observation_space)

# Let's initialize the environment by calling is reset() method. This returns an observation:
env.seed(42)
obs = env.reset()

Action space is: Discrete(2)
Observation space is: Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)


## Define functions
Define some methods for training our agent, plotting etc.


In [7]:
# Map a 4-dimensional state into a state index
# The state space is the four dimensions: x, x_dot, theta, theta_dot
def make_state(observation, DISCRETE_STEPS):
    """ Map a 4-dimensional state into a state index
    """
    low = [-4.8, -10., -0.41888, -10.] # ( changed -0.41 into more correct -0.41888 for 24 deg.)
    high = [4.8, 10., 0.41888, 10.]
    state = 0

    for i in range(4):
        # State variable, projected to the [0, 1] range
        state_variable = (observation[i] - low[i]) / (high[i] - low[i])

        # Discretize. A variable having a value of 0.53 will lead to the integer 5,
        # for instance.
        state_discrete = int(state_variable * DISCRETE_STEPS)
        state_discrete = max(0, state_discrete) # should not be needed
        state_discrete = min(DISCRETE_STEPS-1, state_discrete)

        state *= DISCRETE_STEPS
        state += state_discrete
        # Make state into a 4 "digit" number (between 0 and 9999, if 10 discrete steps)
    return state

In [8]:
def plot_reward(episode_reward):
    '''
        plots episode reward along with
        the running episode mean
    '''
    plt.figure(figsize=(16, 4))
    plt.plot(episode_reward,"b")
    y_av = running_mean(episode_reward, N=100)
    plt.plot(y_av,"r")
    plt.show()

In [9]:
def get_frames(qtable):
    '''
        builds the frame for animation
    '''

    frames = []
    # Animate a run with current qtable
    # env.seed(42)

    # Reset cart and get an initial state
    state = make_state(env.reset(), 12) 

    for step in range(200):
        # From the state find an action
        qvalues = qtable[state]
        action = argmax(qvalues) # greedy action
        # Perform an action to the environment
        next_state, reward, terminate, info = env.step(action)
        state = make_state(next_state, 12)

        if terminate:
            break
        img = env.render(mode="rgb_array")
        frames.append(img)

    return frames

In [75]:
def train_agent_decay(EPISODES, EPSILON, GAMMA, LEARNING_RATE, DISCRETE_STEPS, ALPHA_DECAY, PRINT):
    '''
        trains the agent for the given parameters
        and returns the Q-table and the episode rewards
    '''

    # Initialise statistics to zero
    average_cumulative_reward = 0.0
    episode_reward = np.zeros(EPISODES)

    # Q-table for the discretized states, and two actions
    num_states = DISCRETE_STEPS ** 4
    qtable = [[0., 0.] for state in range(num_states)]
    print('Q-table = %.0f x %.0f' % (len(qtable),len(qtable[0]) ))

    # Decay settings
    MIN_EPSILON = 0.01
    MIN_LR = 0.01

    # Loop over episodes
    for i in range(EPISODES):
        state4D = env.reset()
        state = make_state(state4D, DISCRETE_STEPS)

        terminate = False
        cumulative_reward = 0.0
  
        # decrease epsilon
        EPSILON = max(EPSILON * 0.9995, MIN_EPSILON)

        # decrease learning rate
        alpha = max(LEARNING_RATE / (1 + i * ALPHA_DECAY), MIN_LR)

        for _ in range(200):
            # Compute what the greedy action for the current state is
            qvalues = qtable[state]
            greedy_action = argmax(qvalues)

            # Sometimes, the agent takes a random action, to explore the environment
            if random.random() < EPSILON:
                action = random.randrange(2)
            else:
                action = greedy_action

            # Perform the action
            obs, reward, terminate, info = env.step(action)  # info is ignored
            next_state = make_state(obs, DISCRETE_STEPS)

            if terminate:
                break

            # Update the Q-Table
            td_error = reward + GAMMA * max(qtable[next_state]) - qtable[state][action]
            qtable[state][action] += alpha * td_error

            # Update statistics
            cumulative_reward += reward
            state = next_state

        # store reward for every episode
        episode_reward[i] = cumulative_reward
        
        # Per-episode statistics
        if ((i % 500) == 0 and PRINT):
            print(i, cumulative_reward, sep=',')

    return qtable, episode_reward

## Set random seed
We define a random seed so we can reproduce our results:

In [89]:
env.seed(7)
np.random.seed(7)
tf.random.set_seed(7)

## Implement performance measure
In order to evaluate the model in a better way we can make use of the fact that a total episode reward of 200 is considered as a "completed epsiode". Given that, we can compute the average error, i.e. subtract the episode reward from 200 for each episode, for each model, respectively.

By this performance metric we get a feeling for how well the agent is performing in terms of solving the cartpole problem, as well as a fair measure when comparing models to each other. 

In [90]:
def mean_error(reward):
    '''
        computes the average error
        over all episodes
    '''
    return np.mean(200 - np.array(reward))

## Implement model from Part A
First, we implement our model from Part A but with only 600 episodes and 200 steps per epsiode for it to be comparable with Géron's models and for our performance measure to make sense:

In [91]:
# hyperparameters
EPISODES       = 600    # Number of eposides to run, org. val = 15000
EPSILON        = 0.8    # Chance to explore (take a radom step instead of an "optimal" one), org val = 0.1
GAMMA          = 0.9    # How much previous steps should be rewarded now, discount rate, org val = 0.9
LEARNING_RATE  = 0.5    # Learning rate, org val = 0.1
DISCRETE_STEPS = 12     # Discretization steps per state variable (aviod odd numbers), org val = 10
ALPHA_DECAY    = 0.001  # Learning rate decay
PRINT          = True   # Determine if cumulative episode rewards should be printed

In [92]:
q, r = train_agent_decay(EPISODES, EPSILON, GAMMA, LEARNING_RATE, DISCRETE_STEPS, ALPHA_DECAY, PRINT)

Q-table = 20736 x 2
0,14.0
500,12.0


In [93]:
# save results in dict
results = dict()

In [94]:
# save mean error for my model
results['My model'] = mean_error(r)

## Implement Géron's DQN models
Second, we implement Géron's DQN models. All code provided from Géron's book.

### DQN
Define helper methods:

In [95]:
keras.backend.clear_session()
tf.random.set_seed(7)
np.random.seed(7)

env = gym.make("CartPole-v1")
input_shape = [4] # == env.observation_space.shape
n_outputs = 2 # == env.action_space.n

model = keras.models.Sequential([
    keras.layers.Dense(32, activation="elu", input_shape=input_shape),
    keras.layers.Dense(32, activation="elu"),
    keras.layers.Dense(n_outputs)
])

In [96]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(2)
    else:
        Q_values = model.predict(state[np.newaxis])
        return np.argmax(Q_values[0])

In [97]:
from collections import deque

replay_memory = deque(maxlen=2000)

In [98]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_memory), size=batch_size)
    batch = [replay_memory[index] for index in indices]
    states, actions, rewards, next_states, dones = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(5)]
    return states, actions, rewards, next_states, dones

In [99]:
def play_one_step(env, state, epsilon):
    action = epsilon_greedy_policy(state, epsilon)
    next_state, reward, done, info = env.step(action)
    replay_memory.append((state, action, reward, next_state, done))
    return next_state, reward, done, info

In [100]:
batch_size = 32
discount_rate = 0.95
optimizer = keras.optimizers.Adam(lr=1e-3)
loss_fn = keras.losses.mean_squared_error

def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states, dones = experiences
    next_Q_values = model.predict(next_states)
    max_next_Q_values = np.max(next_Q_values, axis=1)
    target_Q_values = (rewards +
                       (1 - dones) * discount_rate * max_next_Q_values)
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = tf.one_hot(actions, n_outputs)
    with tf.GradientTape() as tape:
        all_Q_values = model(states)
        Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = tf.reduce_mean(loss_fn(target_Q_values, Q_values))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [101]:
rewards = [] 
best_score = 0

for episode in range(600):
    obs = env.reset()    
    for step in range(200):
        epsilon = max(1 - episode / 500, 0.01)
        obs, reward, done, info = play_one_step(env, obs, epsilon)
        if done:
            break
    rewards.append(step) # Not shown in the book
    if step > best_score: # Not shown
        best_weights = model.get_weights() # Not shown
        best_score = step # Not shown
    print("\rEpisode: {}, Steps: {}, eps: {:.3f}".format(episode, step + 1, epsilon), end="") # Not shown
    if episode > 50:
        training_step(batch_size)

model.set_weights(best_weights)

Episode: 599, Steps: 13, eps: 0.010

In [102]:
# save mean error for DQN
results['DQN'] = mean_error(rewards)

### Double DQN

In [103]:
keras.backend.clear_session()
tf.random.set_seed(7)
np.random.seed(7)

model = keras.models.Sequential([
    keras.layers.Dense(32, activation="elu", input_shape=[4]),
    keras.layers.Dense(32, activation="elu"),
    keras.layers.Dense(n_outputs)
])

target = keras.models.clone_model(model)
target.set_weights(model.get_weights())

In [104]:
batch_size = 32
discount_rate = 0.95
optimizer = keras.optimizers.Adam(lr=1e-3)
loss_fn = keras.losses.Huber()

def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states, dones = experiences
    next_Q_values = model.predict(next_states)
    best_next_actions = np.argmax(next_Q_values, axis=1)
    next_mask = tf.one_hot(best_next_actions, n_outputs).numpy()
    next_best_Q_values = (target.predict(next_states) * next_mask).sum(axis=1)
    target_Q_values = (rewards + 
                       (1 - dones) * discount_rate * next_best_Q_values)
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = tf.one_hot(actions, n_outputs)
    with tf.GradientTape() as tape:
        all_Q_values = model(states)
        Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = tf.reduce_mean(loss_fn(target_Q_values, Q_values))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [105]:
replay_memory = deque(maxlen=2000)

In [106]:
rewards = []
best_score = 0

for episode in range(600):
    obs = env.reset()    
    for step in range(200):
        epsilon = max(1 - episode / 500, 0.01)
        obs, reward, done, info = play_one_step(env, obs, epsilon)
        if done:
            break
    rewards.append(step)
    if step > best_score:
        best_weights = model.get_weights()
        best_score = step
    print("\rEpisode: {}, Steps: {}, eps: {:.3f}".format(episode, step + 1, epsilon), end="")
    if episode > 50:
        training_step(batch_size)
    if episode % 50 == 0:
        target.set_weights(model.get_weights())

model.set_weights(best_weights)

Episode: 599, Steps: 11, eps: 0.010

In [107]:
# save mean error for Double DQN
results['Double DQN'] = mean_error(rewards)

### Dueling Double DQN

In [108]:
keras.backend.clear_session()
tf.random.set_seed(7)
np.random.seed(7)

K = keras.backend
input_states = keras.layers.Input(shape=[4])
hidden1 = keras.layers.Dense(32, activation="elu")(input_states)
hidden2 = keras.layers.Dense(32, activation="elu")(hidden1)
state_values = keras.layers.Dense(1)(hidden2)
raw_advantages = keras.layers.Dense(n_outputs)(hidden2)
advantages = raw_advantages - K.max(raw_advantages, axis=1, keepdims=True)
Q_values = state_values + advantages
model = keras.models.Model(inputs=[input_states], outputs=[Q_values])

target = keras.models.clone_model(model)
target.set_weights(model.get_weights())

In [109]:
batch_size = 32
discount_rate = 0.95
optimizer = keras.optimizers.Adam(lr=1e-2)
loss_fn = keras.losses.Huber()

def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states, dones = experiences
    next_Q_values = model.predict(next_states)
    best_next_actions = np.argmax(next_Q_values, axis=1)
    next_mask = tf.one_hot(best_next_actions, n_outputs).numpy()
    next_best_Q_values = (target.predict(next_states) * next_mask).sum(axis=1)
    target_Q_values = (rewards + 
                       (1 - dones) * discount_rate * next_best_Q_values)
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = tf.one_hot(actions, n_outputs)
    with tf.GradientTape() as tape:
        all_Q_values = model(states)
        Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = tf.reduce_mean(loss_fn(target_Q_values, Q_values))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [110]:
replay_memory = deque(maxlen=2000)

In [111]:
rewards = []
best_score = 0

for episode in range(600):
    obs = env.reset()    
    for step in range(200):
        epsilon = max(1 - episode / 500, 0.01)
        obs, reward, done, info = play_one_step(env, obs, epsilon)
        if done:
            break
    rewards.append(step)
    if step > best_score:
        best_weights = model.get_weights()
        best_score = step
    print("\rEpisode: {}, Steps: {}, eps: {:.3f}".format(episode, step + 1, epsilon), end="")
    if episode > 50:
        training_step(batch_size)
    if episode % 200 == 0:
        target.set_weights(model.get_weights())

model.set_weights(best_weights)

Episode: 599, Steps: 120, eps: 0.010

In [112]:
# save mean error for Dueling Double DQN
results['Dueling Double DQN'] = mean_error(rewards)

## Model comparison

In [119]:
# print out result for each model
print('Mean error')
print('---------------')
for mdl, ME in results.items():
    print(mdl + ': ' + str(round(ME, 3)))

Mean error
---------------
My model: 177.093
DQN: 178.205
Double DQN: 170.807
Dueling Double DQN: 177.657


From the mean errors for each model, we can see that the Double DQN performed best, while my model, DQN and Dueling Double DQN performed similarly. We can conclude that we obtained a decent model in Part A, which is clearly comparable with the deep Q-learning models. However, we must note that my model is carefully tuned while the DQN models have not been tuned any further than what Géron perhaps have done. Therefore, it is likely that the deep variations of Q-learning, with a bit of fine-tuning, would outperform my model easily since the capacity of the deep models is higher than in my "simple" implementation. 

Although my model is not the best, the variation in mean error is quite small so from a broader perspective we cannot really distungish between the models. On the other hand, if we consider the lack of tuning in the DQN models we can safely dismiss my model and rely on DQN for further improvements in the cartpole problem.
_____